In [2]:
from train_test_fns import *
import re

In [3]:
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [53]:
def fix_aoki_and_castell(name):
    if name == 'Norichika Aoki':
        return 'Nori Aoki'
    elif name == 'Nicholas Castellanos':
        return 'Nick Castellanos'
    else: return name

In [99]:
def combine_with_lehman_data(df):
    players = pd.read_csv('baseballdatabank-master/core/People.csv')
    #players = players.set_index('playerID')

    drop_cols = ['deathYear','deathMonth','deathDay','deathCountry','deathState','deathCity',
                'birthYear','birthMonth','birthDay','birthCountry','birthState','birthCity',
                'nameGiven','weight','height','bats','throws','finalGame','retroID','bbrefID']
    players = players.drop(drop_cols,axis=1)
    players['fullname'] = players['nameFirst'] + ' ' + players['nameLast']
    players = players.dropna()

    players['fullname'] = players['fullname'].apply(lambda x: ''.join(re.sub(r'[^\w\s]','',x).split(' ')).lower())
    
    batting = pd.read_csv('baseballdatabank-master/core/Batting.csv')
    bats = batting[batting['yearID'] >= 2000]
    
    bat_join = bats.merge(players,how='left',on='playerID')

    keep_cols = ['yearID',
     'G',
     'AB',
     'R',
     'H',
     '2B',
     '3B',
     'HR',
     'RBI',
     'SB',
     'CS',
     'BB',
     'SO',
     'IBB',
     'HBP',
     'SH',
     'SF',
     'GIDP',
     'debut',
     'fullname']

    bat_join = bat_join[keep_cols]

    bat_join.columns = [x.lower() for x in bat_join.columns]

    bat_join = bat_join.groupby(['fullname','yearid','debut'],axis=0)['g','ab','r','h','2b','3b','hr','rbi','sb','cs','bb','so','ibb','hbp','sh','sf','gidp'].sum().reset_index()
    bat_join['str_g'] = bat_join['g'].apply(str)
    bat_join['str_year'] = bat_join['yearid'].apply(str)
    bat_join['name_g_y'] = bat_join['fullname'] + ' ' + bat_join['str_g'] + ' ' + bat_join['str_year']
    
    df['str_g'] = df['g'].apply(str)
    df['str_year'] = df['year'].apply(str)
    df['name'] = df['name'].apply(fix_aoki_and_castell)
    df['name_g_y'] = df['name'].apply(lambda x: ''.join(x.split(' ')).lower()) + ' ' + df['str_g'] + ' ' + df['str_year']
    
    df = df.merge(bat_join,how='left',on='name_g_y')
    
    df = df.dropna()

    df['debut_year'] = df['debut'].apply(lambda x: int(x.split('-')[0]))

    df['years_in_mlb'] = df['year'] - df['debut_year']

    df = df.drop(['g_y','str_g_x','str_g_y','str_year_x','str_year_y','debut','debut_year','yearid','name_g_y','fullname'],axis=1)
    return df




In [103]:
train,test = load_data()

train = engineer_features(train)
test = engineer_features(test)

merged_train = combine_with_lehman_data(train)
merged_test = combine_with_lehman_data(test)

In [105]:
merged_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5377 entries, 0 to 5394
Data columns (total 74 columns):
name            5377 non-null object
age             5377 non-null int64
g_x             5377 non-null int64
pa              5377 non-null int64
rbat            5377 non-null int64
rbaser          5377 non-null int64
rdp             5377 non-null int64
rfield          5377 non-null int64
rpos            5377 non-null int64
raa             5377 non-null int64
waa             5377 non-null float64
rrep            5377 non-null int64
rar             5377 non-null int64
war             5377 non-null float64
waawl%          5377 non-null float64
162wl%          5377 non-null float64
owar            5377 non-null float64
dwar            5377 non-null float64
orar            5377 non-null int64
salary          5377 non-null float64
year            5377 non-null int64
adj_salary      5377 non-null int64
log10_adj       5377 non-null float64
acquired_dr     5377 non-null uint8
acquired_fa 

In [37]:
batting = pd.read_csv('baseballdatabank-master/core/Batting.csv')
bats = batting[batting['yearID'] >= 2000]
#bat_2000_over_100 = bat_2000_over_100.set_index('playerID')

In [15]:
players = pd.read_csv('baseballdatabank-master/core/People.csv')
#players = players.set_index('playerID')

drop_cols = ['deathYear','deathMonth','deathDay','deathCountry','deathState','deathCity',
            'birthYear','birthMonth','birthDay','birthCountry','birthState','birthCity',
            'nameGiven','weight','height','bats','throws','finalGame','retroID','bbrefID']
players = players.drop(drop_cols,axis=1)
players['fullname'] = players['nameFirst'] + ' ' + players['nameLast']
players = players.dropna()

players['fullname'] = players['fullname'].apply(lambda x: ''.join(re.sub(r'[^\w\s]','',x).split(' ')).lower())

In [38]:
bat_join = bats.merge(players,how='left',on='playerID')

keep_cols = ['yearID',
 'G',
 'AB',
 'R',
 'H',
 '2B',
 '3B',
 'HR',
 'RBI',
 'SB',
 'CS',
 'BB',
 'SO',
 'IBB',
 'HBP',
 'SH',
 'SF',
 'GIDP',
 'debut',
 'fullname']

bat_join = bat_join[keep_cols]

bat_join.columns = [x.lower() for x in bat_join.columns]

bat_join = bat_join.groupby(['fullname','yearid','debut'],axis=0)['g','ab','r','h','2b','3b','hr','rbi','sb','cs','bb','so','ibb','hbp','sh','sf','gidp'].sum().reset_index()
bat_join['str_g'] = bat_join['g'].apply(str)
bat_join['str_year'] = bat_join['yearid'].apply(str)
bat_join['name_g_y'] = bat_join['fullname'] + ' ' + bat_join['str_g'] + ' ' + bat_join['str_year']


In [60]:
train['str_g'] = train['g'].apply(str)
train['str_year'] = train['year'].apply(str)
train['name'] = train['name'].apply(fix_aoki_and_castell)
train['name_g_y'] = train['name'].apply(lambda x: ''.join(x.split(' ')).lower()) + ' ' + train['str_g'] + ' ' + train['str_year']

test['str_g'] = test['g'].apply(str)
test['str_year'] = test['year'].apply(str)
test['name'] = test['name'].apply(fix_aoki_and_castell)
test['name_g_y'] = test['name'].apply(lambda x: ''.join(x.split(' ')).lower()) + ' ' + test['str_g'] + ' ' + test['str_year']




In [82]:
merged = train.merge(bat_join,how='left',on='name_g_y')
merged_test = test.merge(bat_join,how='left',on='name_g_y')

merged = merged.dropna()

In [78]:
int(merged['debut'][0].split('-')[0])

1997

In [84]:
merged['debut_year'] = merged['debut'].apply(lambda x: int(x.split('-')[0]))

In [86]:
merged['years_in_mlb'] = merged['year'] - merged['debut_year']

In [91]:
merged = merged.drop(['g_y','str_g_x','str_g_y','str_year_x','str_year_y','debut','debut_year','yearid','name_g_y','fullname'],axis=1)

In [92]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5377 entries, 0 to 5394
Data columns (total 74 columns):
name            5377 non-null object
age             5377 non-null int64
g_x             5377 non-null int64
pa              5377 non-null int64
rbat            5377 non-null int64
rbaser          5377 non-null int64
rdp             5377 non-null int64
rfield          5377 non-null int64
rpos            5377 non-null int64
raa             5377 non-null int64
waa             5377 non-null float64
rrep            5377 non-null int64
rar             5377 non-null int64
war             5377 non-null float64
waawl%          5377 non-null float64
162wl%          5377 non-null float64
owar            5377 non-null float64
dwar            5377 non-null float64
orar            5377 non-null int64
salary          5377 non-null float64
year            5377 non-null int64
adj_salary      5377 non-null int64
log10_adj       5377 non-null float64
acquired_dr     5377 non-null uint8
acquired_fa 